## Using PyGreSQL

Bethany Yollin

In [5]:
import sys
sys.path.append('C:\Python27\ArcGIS10.3\Lib\site-packages')
from pg import DB

In [6]:
# connect to database
db = DB(dbname = 'geog465_bethanyy', host = 'geog-db2.geog.uw.edu', port = 5432, user = 'bethanyy', passwd = 'bethanyy')

In [9]:
# create tables and establish relationships
db.query('''

    CREATE TABLE Incident (
        CAD_ID int  NOT NULL,
        DateTime timestamp  NOT NULL,
        Beat char(2)  NOT NULL,
        Tract decimal(8,2)  NOT NULL,
        Officer int  NOT NULL,
        Block text  NOT NULL,
        Latitude real  NOT NULL,
        Longitude real  NOT NULL,
        Description int  NOT NULL,
        AtScene timestamp  NOT NULL,
        CONSTRAINT Incident_pk PRIMARY KEY (CAD_ID)
    );
    
    CREATE TABLE Crime_Type (
        CRIME_TYPE_ID int  NOT NULL,
        CrimeTypeName varchar(50)  NOT NULL,
        CONSTRAINT Crime_Type_pk PRIMARY KEY (CRIME_TYPE_ID)
    );
    
    CREATE TABLE Crime_Group (
        CRIME_GROUP_ID int  NOT NULL,
        CrimeGroupName varchar(50)  NOT NULL,
        CrimeType int  NOT NULL,
        CONSTRAINT Crime_Group_pk PRIMARY KEY (CRIME_GROUP_ID)
    );
    
    CREATE TABLE Crime_Desc (
        CRIME_DESC_ID int  NOT NULL,
        CrimeDescName varchar(50)  NOT NULL,
        CrimeGroup int  NOT NULL,
        CONSTRAINT Crime_Desc_pk PRIMARY KEY (CRIME_DESC_ID)
    );
    
    ALTER TABLE Incident ADD CONSTRAINT Incident_Crime_Desc
        FOREIGN KEY (Description)
        REFERENCES Crime_Desc (CRIME_DESC_ID)  
        NOT DEFERRABLE 
        INITIALLY IMMEDIATE
    ;
    
    ALTER TABLE Crime_Desc ADD CONSTRAINT Crime_Desc_Crime_Group
        FOREIGN KEY (CrimeGroup)
        REFERENCES Crime_Group (CRIME_GROUP_ID)  
        NOT DEFERRABLE 
        INITIALLY IMMEDIATE
    ;
    
    ALTER TABLE Crime_Group ADD CONSTRAINT Crime_Group_Crime_Type
        FOREIGN KEY (CrimeType)
        REFERENCES Crime_Type (CRIME_TYPE_ID)  
        NOT DEFERRABLE 
        INITIALLY IMMEDIATE
    ;
    
''')

In [10]:
# insert some data
db.insert('crime_type', crime_type_id = 6, crimetypename = 'Property Crime/Theft')

db.insert('crime_group', crime_group_id = 10, crimetype = 6, crimegroupname = 'SHOPLIFTING')

db.insert('crime_desc', crime_desc_id = 101, crimegroup = 10, crimedescname = 'SHOPLIFT')

db.insert('incident', cad_id = 1123325, datetime = '06-17-2015 12:13:00', beat = 'K1',
          tract = 8100.2029, officer = 421, block = '8XX BLOCK OF 3 AV',
          latitude = 47.60455, longitude = -122.333, description = 101,
          atscene = '06-17-2015 12:24:30')

{'atscene': datetime.datetime(2015, 6, 17, 12, 24, 30),
 'beat': 'K1',
 'block': '8XX BLOCK OF 3 AV',
 'cad_id': 1123325,
 'datetime': datetime.datetime(2015, 6, 17, 12, 13),
 'description': 101,
 'latitude': 47.6045,
 'longitude': -122.333,
 'officer': 421,
 'tract': Decimal('8100.20')}

In [11]:
# print table contents       
print db.query('select * from incident')
print db.query('select * from crime_desc')
print db.query('select * from crime_group')
print db.query('select * from crime_type')

cad_id |     datetime      |beat| tract |officer|      block      |latitude|longitude|description|      atscene      
-------+-------------------+----+-------+-------+-----------------+--------+---------+-----------+-------------------
1123325|2015-06-17 12:13:00|K1  |8100.20|    421|8XX BLOCK OF 3 AV| 47.6045| -122.333|        101|2015-06-17 12:24:30
(1 row)
crime_desc_id|crimedescname|crimegroup
-------------+-------------+----------
          101|SHOPLIFT     |        10
(1 row)
crime_group_id|crimegroupname|crimetype
--------------+--------------+---------
            10|SHOPLIFTING   |        6
(1 row)
crime_type_id|   crimetypename    
-------------+--------------------
            6|Property Crime/Theft
(1 row)
